<mark><font color= yellow size=4 face=雅黑>极限学习机(Extreme Learning Machine) ELM<font><mark>

极限学习机(Extreme Learning Machine) ELM，是2006年由黄广斌提出来的求解单隐层神经网络的算法。最大的特点是输入权

值和隐含节点的偏置都是在给定范围内随机生成的，被证实学习效率高且泛化能力强。训练时的主要目的在于输出层的权值求解。

算法步骤

输入 ：样本集，激活函数为，隐藏层节点数

输出：输出权值

步骤1: 随机设置输入权值和隐含节点偏置，

步骤2 : 计算隐藏层输出矩阵，

步骤3: 计算输出矩阵的广义逆矩阵，

步骤4: 计算极速学习机的隐含层到输出层的权值.

Iris 鸢尾花数据集是一个经典数据集，在统计学习和机器学习领域都经常被用作示例。数据集内包含 3 类共 150 条记录，每类各 50 个数据，每条记录都有 4 项特征：花萼长度、花萼宽度、花瓣长度、花瓣宽度，可以通过这4个特征预测鸢尾花卉属于（iris-setosa, iris-versicolour, iris-virginica）中的哪一品种。



In [2]:
import numpy as np
from sklearn.datasets import load_iris  #导入鸢尾花数据集
from sklearn.model_selection import train_test_split  #数据集的分割函数,就是把一组数据分成训练集合测试集
from sklearn.preprocessing import StandardScaler      #数据预处理

#引入包含数据验证方法的包，就是为了来计算准确率的
from sklearn import metrics

class SingeHiddenLayer(object):  #定义单隐藏层这个类
    
    def __init__(self,X,y,num_hidden):#进行初始化，然后进行实例化
        self.data_x = np.atleast_2d(X)       #判断输入训练集是否大于等于二维; 把x_train()取下来
         #numpy.atleast_xd()  atleast_xd 支持将输入数据直接视为 x维
        self.data_y = np.array(y).flatten()  
        #a.flatten()把a放在一维数组中，不写参数默认是“C”，也就是先行后列的方式，也有“F”先列后行的方式； 把 y_train取下来
        self.num_data = len(self.data_x)  #训练数据个数(所有的)
        self.num_feature = self.data_x.shape[1];  #shape[] 读取矩阵的长度，比如shape[0]就是读取矩阵第一维度的长度 (120行，4列，所以shape[0]==120,shapep[1]==4)
        self.num_hidden = num_hidden;  #隐藏层节点个数
        #print(data_x)
        
        #随机生产权重（从-1，到1，生成（num_feature行,num_hidden列））
        self.w = np.random.uniform(-1, 1, (self.num_feature, self.num_hidden)) #权重产生是随机的所有试验具有确定性 
        
        #随机生成偏置，一个隐藏层节点对应一个偏置
        for i in range(self.num_hidden):  #需要得到所有隐藏节点的偏置
            b = np.random.uniform(-0.6, 0.6, (1, self.num_hidden))
            #偏置的范围是从-0.6到 0.6，size是self.num_hidden个  有多少个隐藏神经元就产生多少个偏置
            self.first_b = b  #得到的b是一个数组，不仅仅是一个数  b是一个成员变量不够直接调用
   
        #生成偏置矩阵，以隐藏层节点个数4为行，样本数120为列
        for i in range(self.num_data-1):
            b = np.row_stack((b, self.first_b))  #row_stack 以叠加行的方式填充数组
        self.b = b #实例化
    #定义sigmoid函数  
    def sigmoid(self,x):
        return 1.0 / (1 + np.exp(-x))
        
    def train(self,x_train,y_train,classes):#定义训练方法
        mul = np.dot(self.data_x, self.w)    #输入乘以权重 ,也就是H*beta
        add = mul + self.b                        #加偏置 add表示输出神经元
        H = self.sigmoid(add)                #激活函数  使用激活函数

        H_ = np.linalg.pinv(H)               #求H的广义逆矩阵H_相当论文的于H+
        print(type(H_.shape))                                           #避免使用np
        
        #将只有一列的Label矩阵转换，例如，iris的label中共有三个值，则转换为3列，以行为单位，label值对应位置标记为1，其它位置标记为0
        self.train_y = np.zeros((self.num_data,classes))  #初始化一个120行，3列的全0矩阵
        print(np.shape(self.train_y) )
        
        for i in range(0,self.num_data):
            # print("ruru",y_train[i])
            self.train_y[i,y_train[i]] = 1   #对应位置标记为1  ,y_train干嘛嘞？没整太明白？？？？ onehot编码
        
        self.out_w = np.dot(H_,self.train_y)  #求输出权重  相当于第二个beta
    #下面是对预测的数据进行处理，其基本方法和训练的数据集
    def predict(self,x_test):  #x_test这部分的数据主要是用来进行预测的
        self.t_data = np.atleast_2d(x_test)    #测试数据集
        self.num_tdata = len(self.t_data)      #测试集的样本数
        self.pred_Y = np.zeros((x_test.shape[0]))  #对测试集进行初始化
        print(self.pred_Y)
        #print(len((self.pred_Y.shape)))
        
        b = self.first_b
        
        #扩充偏置矩阵，以隐藏层节点个数4为行，样本数30为列
        for i in range(self.num_tdata-1):
            b = np.row_stack((b, self.first_b))  #以叠加行的方式填充数组
          
        #预测  ，最小beta的输出
        self.pred_Y = np.dot(self.sigmoid(np.dot(self.t_data,self.w)+b),self.out_w) #普通用乘，矩阵用点积
        
        #取输出节点中值最大的类别作为预测值 
        self.predy = []
        for i in self.pred_Y:
            L = i.tolist() #将i这个数组转换成列表 L就是yigelist
            print(L)
           # print(append(L.index(max(L)))
            self.predy.append(L.index(max(L)))   #索引到输出节点中最大的值,索引到的是位置不是数
            #print( self.predy.append(L.index(max(L))) )

    def score(self,y_test):
        print("准确率：")
        #使用准确率方法验证
        print(metrics.accuracy_score(y_true=y_test,y_pred=self.predy)) 
        #调用metrics.accuracy_score方法来计算分类的准确率
        print(y_test)
        print(self.predy)
        print(len(self.predy))
        
stdsc = StandardScaler()   #StandardScaler类,利用接口在训练集上计算均值和标准差，以便于在后续的测试集上进行相同的缩放
iris = load_iris() #进行实例化
x,y = stdsc.fit_transform(iris.data),iris.target   #数据归一化，y表示的就是标签
x_train, x_test, y_train, y_test = train_test_split(x , y, test_size=0.2, random_state=0)
#无论是标签的数据还是训练的数据，都分为训练集和测试集
#x_train，这部分数据干嘛的嘞
ELM = SingeHiddenLayer(x_train,y_train,4)   #训练数据集，训练集的label，隐藏层节点个数
ELM.train(x_train,y_train,3) #y_train表示训练集的label
ELM.predict(x_test)#x_test这个数据集就是用来预测的
ELM.score(y_test)

<class 'tuple'>
(120, 3)
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0.]
[-0.47566036894241037, 0.16930476660754676, 1.2284750547317214]
[-0.12091367378267054, 0.789834694770149, 0.31257377581457624]
[0.82190174617096, -0.1719595533102497, 0.3861307430919268]
[0.21391620985196227, 0.525739360872818, 0.36182551785285244]
[0.8788356340930753, 0.19339130158899034, -0.08584972247518573]
[0.048710756518612314, 0.03742834387654029, 0.9311025391819346]
[0.7682093607632775, 0.06662323014274636, 0.1216154165727809]
[0.3191231436429438, 0.29455011370814643, 0.40092196508200945]
[0.134163693204509, 0.48200795560388504, 0.3999426801524361]
[0.13011964236260298, 0.41504721870740013, 0.4321211741746151]
[0.332218399125687, 0.7821863584171718, 0.013882947098076394]
[0.3922471430981581, 0.22080990071355672, 0.39961454338286495]
[0.4256544518210695, 0.596944556725744, 0.052972866127276884]
[0.052617871531488385, 0.4231826776150212, 0.5157622105508535]
[0.339

<mark><font color= yellow size=4 face=雅黑>Iris 鸢尾花数据集<font><mark>


Iris 鸢尾花数据集是一个经典数据集，在统计学习和机器学习领域都经常被用作示例。数据集内包含 3 类共 150 条记录，每类各 50 个数据，每条记录都有 4 项特征：花萼长度、花萼宽度、花瓣长度、花瓣宽度，可以通过这4个特征预测鸢尾花卉属于（iris-setosa, iris-versicolour, iris-virginica）中的哪一品种。

In [3]:
print(y)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]


In [4]:
print(len(y))

150


In [67]:
print(iris)

{'data': array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.4, 3.7, 1.5, 0.2],
       [4.8, 3.4, 1.6, 0.2],
       [4.8, 3. , 1.4, 0.1],
       [4.3, 3. , 1.1, 0.1],
       [5.8, 4. , 1.2, 0.2],
       [5.7, 4.4, 1.5, 0.4],
       [5.4, 3.9, 1.3, 0.4],
       [5.1, 3.5, 1.4, 0.3],
       [5.7, 3.8, 1.7, 0.3],
       [5.1, 3.8, 1.5, 0.3],
       [5.4, 3.4, 1.7, 0.2],
       [5.1, 3.7, 1.5, 0.4],
       [4.6, 3.6, 1. , 0.2],
       [5.1, 3.3, 1.7, 0.5],
       [4.8, 3.4, 1.9, 0.2],
       [5. , 3. , 1.6, 0.2],
       [5. , 3.4, 1.6, 0.4],
       [5.2, 3.5, 1.5, 0.2],
       [5.2, 3.4, 1.4, 0.2],
       [4.7, 3.2, 1.6, 0.2],
       [4.8, 3.1, 1.6, 0.2],
       [5.4, 3.4, 1.5, 0.4],
       [5.2, 4.1, 1.5, 0.1],
       [5.5, 4.2, 1.4, 0.2],
     

In [71]:
 print(len(iris))

6


In [13]:
print(x)
print(type(x))

[[-9.00681170e-01  1.01900435e+00 -1.34022653e+00 -1.31544430e+00]
 [-1.14301691e+00 -1.31979479e-01 -1.34022653e+00 -1.31544430e+00]
 [-1.38535265e+00  3.28414053e-01 -1.39706395e+00 -1.31544430e+00]
 [-1.50652052e+00  9.82172869e-02 -1.28338910e+00 -1.31544430e+00]
 [-1.02184904e+00  1.24920112e+00 -1.34022653e+00 -1.31544430e+00]
 [-5.37177559e-01  1.93979142e+00 -1.16971425e+00 -1.05217993e+00]
 [-1.50652052e+00  7.88807586e-01 -1.34022653e+00 -1.18381211e+00]
 [-1.02184904e+00  7.88807586e-01 -1.28338910e+00 -1.31544430e+00]
 [-1.74885626e+00 -3.62176246e-01 -1.34022653e+00 -1.31544430e+00]
 [-1.14301691e+00  9.82172869e-02 -1.28338910e+00 -1.44707648e+00]
 [-5.37177559e-01  1.47939788e+00 -1.28338910e+00 -1.31544430e+00]
 [-1.26418478e+00  7.88807586e-01 -1.22655167e+00 -1.31544430e+00]
 [-1.26418478e+00 -1.31979479e-01 -1.34022653e+00 -1.44707648e+00]
 [-1.87002413e+00 -1.31979479e-01 -1.51073881e+00 -1.44707648e+00]
 [-5.25060772e-02  2.16998818e+00 -1.45390138e+00 -1.31544430e

In [62]:
print(y)
print(type(y.shape))

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
<class 'tuple'>


In [7]:
print(train_y)

NameError: name 'train_y' is not defined

In [55]:
print(x)
print(np.shape(x))

[[-9.00681170e-01  1.01900435e+00 -1.34022653e+00 -1.31544430e+00]
 [-1.14301691e+00 -1.31979479e-01 -1.34022653e+00 -1.31544430e+00]
 [-1.38535265e+00  3.28414053e-01 -1.39706395e+00 -1.31544430e+00]
 [-1.50652052e+00  9.82172869e-02 -1.28338910e+00 -1.31544430e+00]
 [-1.02184904e+00  1.24920112e+00 -1.34022653e+00 -1.31544430e+00]
 [-5.37177559e-01  1.93979142e+00 -1.16971425e+00 -1.05217993e+00]
 [-1.50652052e+00  7.88807586e-01 -1.34022653e+00 -1.18381211e+00]
 [-1.02184904e+00  7.88807586e-01 -1.28338910e+00 -1.31544430e+00]
 [-1.74885626e+00 -3.62176246e-01 -1.34022653e+00 -1.31544430e+00]
 [-1.14301691e+00  9.82172869e-02 -1.28338910e+00 -1.44707648e+00]
 [-5.37177559e-01  1.47939788e+00 -1.28338910e+00 -1.31544430e+00]
 [-1.26418478e+00  7.88807586e-01 -1.22655167e+00 -1.31544430e+00]
 [-1.26418478e+00 -1.31979479e-01 -1.34022653e+00 -1.44707648e+00]
 [-1.87002413e+00 -1.31979479e-01 -1.51073881e+00 -1.44707648e+00]
 [-5.25060772e-02  2.16998818e+00 -1.45390138e+00 -1.31544430e

NameError: name 'self' is not defined

In [3]:
print(y_test)

[2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0]


In [5]:
print(y_pred)

NameError: name 'y_pred' is not defined

In [43]:
print(iris.data)

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]
 [5.4 3.9 1.7 0.4]
 [4.6 3.4 1.4 0.3]
 [5.  3.4 1.5 0.2]
 [4.4 2.9 1.4 0.2]
 [4.9 3.1 1.5 0.1]
 [5.4 3.7 1.5 0.2]
 [4.8 3.4 1.6 0.2]
 [4.8 3.  1.4 0.1]
 [4.3 3.  1.1 0.1]
 [5.8 4.  1.2 0.2]
 [5.7 4.4 1.5 0.4]
 [5.4 3.9 1.3 0.4]
 [5.1 3.5 1.4 0.3]
 [5.7 3.8 1.7 0.3]
 [5.1 3.8 1.5 0.3]
 [5.4 3.4 1.7 0.2]
 [5.1 3.7 1.5 0.4]
 [4.6 3.6 1.  0.2]
 [5.1 3.3 1.7 0.5]
 [4.8 3.4 1.9 0.2]
 [5.  3.  1.6 0.2]
 [5.  3.4 1.6 0.4]
 [5.2 3.5 1.5 0.2]
 [5.2 3.4 1.4 0.2]
 [4.7 3.2 1.6 0.2]
 [4.8 3.1 1.6 0.2]
 [5.4 3.4 1.5 0.4]
 [5.2 4.1 1.5 0.1]
 [5.5 4.2 1.4 0.2]
 [4.9 3.1 1.5 0.2]
 [5.  3.2 1.2 0.2]
 [5.5 3.5 1.3 0.2]
 [4.9 3.6 1.4 0.1]
 [4.4 3.  1.3 0.2]
 [5.1 3.4 1.5 0.2]
 [5.  3.5 1.3 0.3]
 [4.5 2.3 1.3 0.3]
 [4.4 3.2 1.3 0.2]
 [5.  3.5 1.6 0.6]
 [5.1 3.8 1.9 0.4]
 [4.8 3.  1.4 0.3]
 [5.1 3.8 1.6 0.2]
 [4.6 3.2 1.4 0.2]
 [5.3 3.7 1.5 0.2]
 [5.  3.3 1.4 0.2]
 [7.  3.2 4.7 1.4]
 [6.4 3.2 4.5 1.5]
 [6.9 3.1 4.

In [58]:
print(type(iris).shape())

AttributeError: type object 'Bunch' has no attribute 'shape'

<mark><font color= yellow size=5 face=雅黑>将simoid替换为tanh函数<font><mark>

In [82]:
import numpy as np
from sklearn.datasets import load_iris  #导入鸢尾花数据集
from sklearn.model_selection import train_test_split  #数据集的分割函数
from sklearn.preprocessing import StandardScaler      #数据预处理

#引入包含数据验证方法的包
from sklearn import metrics

class SingeHiddenLayer(object):  #定义单隐藏层这个类
    
    def __init__(self,X,y,num_hidden):#进行初始化
        self.data_x = np.atleast_2d(X)       #判断输入训练集是否大于等于二维; 把x_train()取下来
        self.data_y = np.array(y).flatten()  #a.flatten()把a放在一维数组中，不写参数默认是“C”，也就是先行后列的方式，也有“F”先列后行的方式； 把 y_train取下来
        self.num_data = len(self.data_x)  #训练数据个数
        self.num_feature = self.data_x.shape[1];  #shape[] 读取矩阵的长度，比如shape[0]就是读取矩阵第一维度的长度 (120行，4列，所以shape[0]==120,shapep[1]==4)
        self.num_hidden = num_hidden;  #隐藏层节点个数
        
        #随机生产权重（从-1，到1，生成（num_feature行,num_hidden列））
        self.w = np.random.uniform(-1, 1, (self.num_feature, self.num_hidden)) 
        
        #随机生成偏置，一个隐藏层节点对应一个偏置
       # for i in range(self.num_hidden):
        b = np.random.uniform(-0.6, 0.6, (1, self.num_hidden))
        self.first_b = b
   
        #生成偏置矩阵，以隐藏层节点个数4为行，样本数120为列
        for i in range(self.num_data-1):
            b = np.row_stack((b, self.first_b))  #row_stack 以叠加行的方式填充数组
        self.b = b
    #定义sigmoid函数  
    def tanh(self,x):
        return 2*1.0 / (1 + np.exp(-x))-1
        
    def train(self,x_train,y_train,classes):
        mul = np.dot(self.data_x, self.w)    #输入乘以权重 
        add = mul + self.b                        #加偏置
        H = self.tanh(add)                #激活函数

        H_ = np.linalg.pinv(H)               #求广义逆矩阵
        #print(type(H_.shape))
        
        #将只有一列的Label矩阵转换，例如，iris的label中共有三个值，则转换为3列，以行为单位，label值对应位置标记为1，其它位置标记为0
        self.train_y = np.zeros((self.num_data,classes))  #初始化一个120行，3列的全0矩阵
        for i in range(0,self.num_data):
            self.train_y[i,y_train[i]] = 1   #对应位置标记为1
        
        self.out_w = np.dot(H_,self.train_y)  #求输出权重
        
    def predict(self,x_test):
        self.t_data = np.atleast_2d(x_test)    #测试数据集
        self.num_tdata = len(self.t_data)      #测试集的样本数
        self.pred_Y = np.zeros((x_test.shape[0]))  #对测试集进行初始化
        
        b = self.first_b
        
        #扩充偏置矩阵，以隐藏层节点个数4为行，样本数30为列
        for i in range(self.num_tdata-1):
            b = np.row_stack((b, self.first_b))  #以叠加行的方式填充数组
          
         #预测  
        self.pred_Y = np.dot(self.tanh(np.dot(self.t_data,self.w)+b),self.out_w)
        
        #取输出节点中值最大的类别作为预测值 
        self.predy = []
        for i in self.pred_Y:
            L = i.tolist()
            self.predy.append(L.index(max(L)))  

    def score(self,y_test):
        print("准确率：")
        #使用准确率方法验证
        print(metrics.accuracy_score(y_true=y_test,y_pred=self.predy))
        
stdsc = StandardScaler()   #StandardScaler类,利用接口在训练集上计算均值和标准差，以便于在后续的测试集上进行相同的缩放
iris = load_iris()
x,y = stdsc.fit_transform(iris.data),iris.target   #数据归一化，y表示的就是标签
x_train, x_test, y_train, y_test = train_test_split(x , y, test_size=0.2, random_state=0)

ELM = SingeHiddenLayer(x_train,y_train,4)   #训练数据集，训练集的label，隐藏层节点个数
ELM.train(x_train,y_train,3) #y_train表示训练集的label
ELM.predict(x_test)
ELM.score(y_test)

准确率：
0.7


<mark><font color= yellow size=5 face=雅黑>将simoid替换为Relu函数<font><mark>

In [77]:
import numpy as np
from sklearn.datasets import load_iris  #导入鸢尾花数据集
from sklearn.model_selection import train_test_split  #数据集的分割函数
from sklearn.preprocessing import StandardScaler      #数据预处理

#引入包含数据验证方法的包
from sklearn import metrics

class SingeHiddenLayer(object):  #定义单隐藏层这个类
    
    def __init__(self,X,y,num_hidden):#进行初始化
        self.data_x = np.atleast_2d(X)       #判断输入训练集是否大于等于二维; 把x_train()取下来
        self.data_y = np.array(y).flatten()  #a.flatten()把a放在一维数组中，不写参数默认是“C”，也就是先行后列的方式，也有“F”先列后行的方式； 把 y_train取下来
        self.num_data = len(self.data_x)  #训练数据个数
        self.num_feature = self.data_x.shape[1];  #shape[] 读取矩阵的长度，比如shape[0]就是读取矩阵第一维度的长度 (120行，4列，所以shape[0]==120,shapep[1]==4)
        self.num_hidden = num_hidden;  #隐藏层节点个数
        
        #随机生产权重（从-1，到1，生成（num_feature行,num_hidden列））
        self.w = np.random.uniform(-1, 1, (self.num_feature, self.num_hidden)) 
        
        #随机生成偏置，一个隐藏层节点对应一个偏置
        for i in range(self.num_hidden):
            b = np.random.uniform(-0.6, 0.6, (1, self.num_hidden))
            self.first_b = b
   
        #生成偏置矩阵，以隐藏层节点个数4为行，样本数120为列
        for i in range(self.num_data-1):
            b = np.row_stack((b, self.first_b))  #row_stack 以叠加行的方式填充数组
        self.b = b
    #定义sigmoid函数  
    def Relu(self,x):
        return np.maximum(x, 0)

        
    def train(self,x_train,y_train,classes):
        mul = np.dot(self.data_x, self.w)    #输入乘以权重 
        add = mul + self.b                        #加偏置
        H = self.Relu(add)                #激活函数

        H_ = np.linalg.pinv(H)               #求广义逆矩阵
        #print(type(H_.shape))
        
        #将只有一列的Label矩阵转换，例如，iris的label中共有三个值，则转换为3列，以行为单位，label值对应位置标记为1，其它位置标记为0
        self.train_y = np.zeros((self.num_data,classes))  #初始化一个120行，3列的全0矩阵
        for i in range(0,self.num_data):
            self.train_y[i,y_train[i]] = 1   #对应位置标记为1
        
        self.out_w = np.dot(H_,self.train_y)  #求输出权重
        
    def predict(self,x_test):
        self.t_data = np.atleast_2d(x_test)    #测试数据集
        self.num_tdata = len(self.t_data)      #测试集的样本数
        self.pred_Y = np.zeros((x_test.shape[0]))  #对测试集进行初始化
        
        b = self.first_b
        
        #扩充偏置矩阵，以隐藏层节点个数4为行，样本数30为列
        for i in range(self.num_tdata-1):
            b = np.row_stack((b, self.first_b))  #以叠加行的方式填充数组
          
         #预测  
        self.pred_Y = np.dot(self.Relu(np.dot(self.t_data,self.w)+b),self.out_w)
        
        #取输出节点中值最大的类别作为预测值 
        self.predy = []
        for i in self.pred_Y:
            L = i.tolist()
            self.predy.append(L.index(max(L)))  

    def score(self,y_test):
        print("准确率：")
        #使用准确率方法验证
        print(metrics.accuracy_score(y_true=y_test,y_pred=self.predy))
        
stdsc = StandardScaler()   #StandardScaler类,利用接口在训练集上计算均值和标准差，以便于在后续的测试集上进行相同的缩放
iris = load_iris()
x,y = stdsc.fit_transform(iris.data),iris.target   #数据归一化，y表示的就是标签
x_train, x_test, y_train, y_test = train_test_split(x , y, test_size=0.2, random_state=0)

ELM = SingeHiddenLayer(x_train,y_train,4)   #训练数据集，训练集的label，隐藏层节点个数
ELM.train(x_train,y_train,3) #y_train表示训练集的label
ELM.predict(x_test)
ELM.score(y_test)

准确率：
0.8666666666666667
